In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
import plotly.express as px


In [ ]:
#Coleta dos bancos de dados de usuários e projetos

df = pd.read_csv('CSVs/Dados Finais/dadosFinais2.csv', sep=';')
lista = pd.read_csv('CSVs/Dados Brutos/projetos.csv', sep='\t').sort_values(['Projeto'], ascending=True).reset_index(drop=True)

In [ ]:
#Cria uma lista com todos os projetos
projetos = []

for x in lista['Projeto']:
    projetos.append(x)

In [ ]:
#Função que cria um gráfico de contribuições feitas por cada país
def graficopais(projeto):
    #If responsavél por mudar a organização do banco de dados caso seja geral ou especifico
    if projeto == "Geral":
        paises = df.groupby(['País']).sum().reset_index().sort_values('Contribuições', ascending=False).reset_index(drop=True)
    else:
        paises = df.groupby(['País', 'Projeto']).sum().reset_index().sort_values(['Projeto', 'Contribuições'], ascending=False).reset_index(drop=True)

    #Variavéis que vão armazenar os nomes e contribuições de cada país
    nomes_paises = []
    n_contrib = []
    contador = 0                        #O contador serve para mostrar apenas os países que mais contribuiram

    #For responsavel por percorrer o tamanho da variavel paises
    for x in range(len(paises)):
        #Aqui é separado o projeto escolhido ou se todos devem ser considerados
        if projeto == "Geral" or paises['Projeto'][x] == projeto:
            #Esse segunfo If ignora as contribuições nulas ou países que não contribuiram o bastante
            if paises['País'][x] != "<null>" and contador < 10:
                #Atribuição dos valores e incremento do contador
                nomes_paises.append(paises['País'][x])
                n_contrib.append(paises['Contribuições'][x])
                contador += 1

    #Grafico sendo feito, com as suas opções estéticas, como linhas e orientação dos nomes
    plt.bar(nomes_paises, n_contrib, zorder=2)
    plt.title("Contribuições por País - " + projeto)
    plt.xlabel("Países")
    plt.ylabel("Número de contribuições")
    plt.grid(axis='y', linestyle='--', zorder=1)
    plt.xticks(nomes_paises, rotation='vertical')
    plt.show()

In [ ]:
interact(graficopais, projeto=projetos);

In [ ]:
#Função que cria um gráfico da porcentagem de perfis abandonados
def graficoabandonos(projeto):
    #If responsavél por mudar a organização do banco de dados caso seja geral ou especifico
    if projeto == "Geral":
        abandonos = df.groupby(['Atualizado_em_Ano']).size().reset_index(name='Quantidade').sort_values('Atualizado_em_Ano', ascending=False).reset_index(drop=True)
    else:
        abandonos = df.groupby(['Projeto', 'Atualizado_em_Ano']).size().reset_index(name='Quantidade').sort_values(['Projeto', 'Atualizado_em_Ano'], ascending=False).reset_index(drop=True)

    #Variavéis que vão armazenar os anos e quantidades
    ano = []
    quantidade = []
    outros = 0                  #Serve para somar a quantidade de anos além de 2025 e 2024 devido à baixa quantidade

    #For responsavél por percorrer o tamanho da variavél abandonos
    for x in range(len(abandonos)):
        #Aqui é separado o projeto escolhido ou se todos devem ser considerados
        if projeto == "Geral" or abandonos['Projeto'][x] == projeto:
            #Inclui os anos de 2025 e 2024, para além deles, todos são todos somados juntos
            if abandonos['Atualizado_em_Ano'][x] == 2025 or abandonos['Atualizado_em_Ano'][x] == 2024:
                ano.append(abandonos['Atualizado_em_Ano'][x])
                quantidade.append(abandonos['Quantidade'][x])
            else:
                outros = outros + abandonos['Quantidade'][x]

    #Criação do grafico de pizza com preferências estéticas
    ano.append('Outros Anos')
    quantidade.append(outros)
    plt.title("Último Ano de Atividade - " + projeto)
    plt.pie(
        quantidade,
        labels=ano,
        autopct='%1.1f%%',
    )
    plt.show()

In [ ]:
interact(graficoabandonos, projeto=projetos);

In [ ]:
#Função que cria um gráfico das idades das contas que mais contribuiram
def graficocriacao(projeto):
    #If responsavél por mudar a organização do banco de dados caso seja geral ou especifico
    if projeto == "Geral":
        criacao = df.groupby(['Criado_em_Ano']).size().reset_index(name='Quantidade').sort_values('Criado_em_Ano', ascending=False).reset_index(drop=True)
    else:
        criacao = df.groupby(['Projeto', 'Criado_em_Ano']).size().reset_index(name='Quantidade').sort_values('Projeto', ascending=False).reset_index(drop=True)

    #Variaveis para armazenar os anos e as quantidades
    ano = []
    quantidade = []

    #For responsavél por percorrer o tamanho da variavél criacao
    for x in range(len(criacao)):
        #Aqui é separado o projeto escolhido ou se todos devem ser considerados
        if projeto == "Geral" or criacao['Projeto'][x] == projeto:
            ano.append(criacao['Criado_em_Ano'][x])
            quantidade.append(criacao['Quantidade'][x])

    #Grafico sendo feito, com as suas opções estéticas, como linhas, titulo e etc
    plt.title("Idade das Contas - " + projeto)
    plt.bar(ano, quantidade, zorder=2)
    plt.xlabel("Ano de Criação")
    plt.ylabel("Número de Contas")
    plt.grid(axis='y', linestyle='--', zorder=1)
    plt.show()

In [ ]:
interact(graficocriacao, projeto=projetos);

In [ ]:
#Função que cria um gráfico das contribuições por idade das contas que mais contribuiram
def graficocontribuicoes(projeto):
    #If responsavél por mudar a organização do banco de dados caso seja geral ou especifico
    if projeto == "Geral":
        contribuicoes = df.groupby(['Criado_em_Ano', 'Contribuições']).size().reset_index(name='Quantidade')
    else:
        contribuicoes = df.groupby(['Criado_em_Ano', 'Contribuições', 'Projeto']).size().reset_index(
            name='Quantidade').sort_values(by=['Projeto', 'Criado_em_Ano'], ascending=False).reset_index(drop=True)

    #Variaveis que vão armazenar o ano de criação, a quantidade de contribuições feita por cada, o pai de cada valor no gráfico e uma variavel auxiliar
    ano = []
    quantidade = []
    pais = []
    aux = 0

    #Começamos gerando a raiz da árvore, definindo que ela é a raiz e seu valor como 0
    ano.append("Contribuições por Idade de Conta - " + projeto)
    pais.append("")
    quantidade.append(0)

    #For responsavél por percorrer o tamanho da variavél contribuicoes
    for x in range(len(contribuicoes)):
        #Aqui é separado o projeto escolhido ou se todos devem ser considerados
        if projeto == "Geral" or contribuicoes['Projeto'][x] == projeto:
            #Neste ponto, caso o ano seja diferente do anterior, ele agrega o novo ano de criação, diz de quem ele é filho e coloca seu valor
            if x != 0 and contribuicoes['Criado_em_Ano'][x] != contribuicoes['Criado_em_Ano'][x - 1]:
                ano.append(contribuicoes['Criado_em_Ano'][x])
                pais.append("Contribuições por Idade de Conta - " + projeto)
                quantidade.append(int(contribuicoes['Quantidade'][x]) * int(contribuicoes['Contribuições'][x]))
                aux += 1
            #Caso seja o primeiro registro, ele marca como os demais
            elif x == 0:
                ano.append(contribuicoes['Criado_em_Ano'][x])
                quantidade.append(int(contribuicoes['Quantidade'][x]) * int(contribuicoes['Contribuições'][x]))
                pais.append("Contribuições por Idade de Conta - " + projeto)
                aux += 1
            #Caso não seja um primeiro registro ou mudança de ano, ele soma o valor na quantidade preexistente
            else:
                quantidade[aux] = quantidade[aux] + (int(contribuicoes['Quantidade'][x]) * int(contribuicoes['Contribuições'][x]))
                #É necessario multiplicar a Quantidade pelas Contribuições devido à organização da consulta que pode marcar 2 contas de 2025 fizeram 3 contribuições, ou seja, que contas de 2025 fizeram 6 contribuições, como pode haver diversas delas, é necessário somar com as que já foram contabilizadas

            #Aqui é somado todas as contribuições ao pai para poder ser facilmente visualizado no gráfico
            quantidade[0] = quantidade[0] + (int(contribuicoes['Quantidade'][x]) * int(contribuicoes['Contribuições'][x]))

    #Criação do Treemap
    fig = px.treemap(
        names=ano, values=quantidade, parents=pais,
        title=("Contribuições por Idade de Conta - " + projeto),
        branchvalues='total'
    )
    fig.update_traces(root_color="lightgrey")
    fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
    fig.show()

In [ ]:
interact(graficocontribuicoes, projeto=projetos);